Create connection with a container in azure storage account

In [ ]:
dbutils.fs.mount(
  source = "wasbs://mycontainer@mystorageaccount.blob.core.windows.net",
  mount_point = "/mnt/blobstorages",
  extra_configs = {"fs.azure.account.key.mystorageaccount.blob.core.windows.net":"keyofmystorageaccount"})

Import librairies

In [ ]:
import yahoo_fin.stock_info as si
from yahoo_fin import options
import pandas as pd
import yfinance as yf
import copy
import finnhub

Import the original file from Azure container

In [ ]:
df_portfolio_tickers_excel=pd.read_excel("/dbfs/mnt/blobstorages/Client_portfolio_source.xlsx",sheet_name="PTF", usecols="A:F")

Clean the original file

In [ ]:
#take a copy from the original file
df_portfolio_tickers=copy.deepcopy(df_portfolio_tickers_excel)

#only select the data needed
df_portfolio_tickers=df_portfolio_tickers.iloc[4:106,:]

#index the dataframe to process the dataframe in an easier way
df_portfolio_tickers=df_portfolio_tickers.reset_index(level=0)
df_portfolio_tickers=df_portfolio_tickers.drop('index',axis=1)

#put understanding name for the columns
new_columns_name = ['Name', 'ticker', 'buyPrice','buyQuantity','buyValuation','buyDate']
for i,col in enumerate(df_portfolio_tickers.columns):
    df_portfolio_tickers.rename(columns={col:new_columns_name[i]}, inplace=True)

#df replace name of company which have new names
df_portfolio_tickers['Name']=df_portfolio_tickers['Name'].apply(str)
df_portfolio_tickers=df_portfolio_tickers.replace(to_replace="BHTG",value="RENO")
df_portfolio_tickers=df_portfolio_tickers.replace(to_replace="GBOX",value="RVYL")
df_portfolio_tickers=df_portfolio_tickers.replace(to_replace="Greenbox POS",value="Ryvyl Inc.")
df_portfolio_tickers=df_portfolio_tickers.replace(to_replace="BioHiTech Global, Inc.",value="Renovare Environmental,Inc.")


Determine which companies are still listed on the stock exchange

In [ ]:
#create a list to run the function afterward
list_portfolio_tickers=df_portfolio_tickers['ticker'].tolist()

# create dataframes in order to sort tickers
df_tickers_no_exist=pd.DataFrame()
df_tickers_exist=pd.DataFrame()

#In order to know the last day of stock exchange we take last day of the function yf.download with Apple as a reference
stock_data = yf.download('AAPL')
last_day_index = stock_data.tail(1).index

# with a loop check for each tickers if it display a result with the yf.download()function and check if the last day of the result is the last day of the stock exchange
for ticker in list_portfolio_tickers:
    if yf.download(ticker).shape[0] > 2 and yf.download(ticker).tail(1).index==last_day_index:
         df_tickers_exist = df_tickers_exist.append({'ticker': ticker}, ignore_index=True)
    else:
         df_tickers_no_exist = df_tickers_no_exist.append({'ticker': ticker}, ignore_index=True)

Obtaining companies which are still listed on stock exchange

In [ ]:
#for tickers which still exist
#copy all the company portfolio
df_info_ticker_exist=copy.deepcopy(df_portfolio_tickers)

#Removing the rows with tickers which are contained in the df containing ticker which don't exist anymore
cond_ticker_no_exist=df_info_ticker_exist['ticker'].isin(df_tickers_no_exist['ticker'])
df_info_ticker_exist.drop(df_info_ticker_exist [cond_ticker_no_exist].index,inplace = True)

Obtaining companies which are no longer listed on stock exchange

In [ ]:
#for tickers which don't exist anymore
#copy all the company portfolio
df_info_ticker_no_exist=copy.deepcopy(df_portfolio_tickers)

#Removing the rows with tickers which are contained in the df containing ticker which exist
cond_ticker_exist=df_info_ticker_no_exist['ticker'].isin(df_tickers_exist['ticker'])
df_info_ticker_no_exist.drop(df_info_ticker_no_exist[cond_ticker_exist].index,inplace=True)

Obtaining the stock exchange historic data of companies still listed

In [ ]:
#create a list with tickers which exist inorderto run the function afterward
list_portfolio_ticker_exist=df_info_ticker_exist['ticker'].tolist()

#apply the fonction to each variable and concatenate the results
df_historic = pd.concat([si.get_data(ticker,start_date="01-01-2016")for ticker in list_portfolio_ticker_exist],axis=0)

#index the datafrme to process the dataframe
df_historic=df_historic.reset_index(level=0)

#put the good name for the column and in the good format
df_historic=df_historic.rename(columns={'index':'date'})
df_historic['date'] = pd.to_datetime(df_historic['date'], format='%d-%m-%Y')

Upload the results in a csv format to a container in azure storage account

In [ ]:
df_historic.to_csv('/dbfs/mnt/blobstorages/historic_ticker_exist.csv')
df_info_ticker_exist.to_csv('/dbfs/mnt/blobstorages/info_ticker_existt.csv')
df_info_ticker_no_exist.to_csv('/dbfs/mnt/blobstorages/info_ticker_no_exist.csv')

Obtaining info regarding companies still listed  thanks to finnhub API 

In [ ]:
#input the API key
finnhub_client = finnhub.Client(api_key="myAPI")

#create the dataframe in which the result will be saved
df_company_profile=pd.DataFrame()

#create a loop to apply the fonction at each variable and concatenate each results
for ticker in list_portfolio_ticker_exist:
    one_company_profile=pd.DataFrame(finnhub_client.company_profile2(symbol=ticker),index=[0])
    df_company_profile=pd.concat((df_company_profile,one_company_profile),axis=0)

Upload the results in a csv format to a container in azure storage account

In [ ]:
df_company_profile.to_csv('/dbfs/mnt/blobstorages/company_profile_50.csv')

Obtaining recommendation trend regarding companies still listed  thanks to finnhub API 

In [ ]:
#create the dataframe in which the result will be saved
df_company_recommendation_trend=pd.DataFrame()

#create a loop to apply the fonction at each variable and concatenate each results
for ticker in list_portfolio_ticker_exist:
    df_company_recommendation_trend=pd.concat((df_company_recommendation_trend,
                                               pd.DataFrame(finnhub_client.recommendation_trends(ticker)[0],index=[0]),
                                               pd.DataFrame(finnhub_client.recommendation_trends(ticker)[1],index=[0]),
                                               pd.DataFrame(finnhub_client.recommendation_trends(ticker)[2],index=[0]),
                                               pd.DataFrame(finnhub_client.recommendation_trends(ticker)[3],index=[0])),axis=0)

Upload the results in a csv format to a container in azure storage account

In [ ]:
df_company_recommendation_trend.to_csv('/dbfs/mnt/blobstorages/company_recommendation_trend_39_50.csv')